In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install traker

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn as nn
from torchvision.datasets import Food101, ImageFolder
from trak import TRAKer
from trak import modelout_functions
from collections.abc import Iterable

from src.train import train_model
from src.early_stopping import EarlyStopping


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

In [3]:
def preprocess_image(image):
    image = transforms.functional.pil_to_tensor(image)
    processed_image = processor.preprocess(image)["pixel_values"][0]
    return torch.from_numpy(processed_image)

num_classes = 10

train_dataset = Food101("data/food-101", split="train", transform=preprocess_image, download=False)
test_dataset = Food101("data/food-101", split="test", transform=preprocess_image, download=False)

In [4]:
filtered_dataset = ImageFolder("data/food-101/food-101/images", transform=preprocess_image)

# bruschetta, garlic bread, grilled salmon, omelette, pancakes,
# pizza, porkchop, spaghetti bolognese, spaghetti carbonara, steak
chosen_indices = [10, 46, 50, 67, 72, 76, 77, 90, 91, 93]
new_class_idx_mapping = {k: v for k, v in zip(chosen_indices, range(num_classes))}

filtered_dataset.classes = [filtered_dataset.classes[i] for i in chosen_indices]
filtered_dataset.class_to_idx = {k: i for i, k in enumerate(filtered_dataset.classes)}

print(len(filtered_dataset))

filtered_dataset.samples = list(filter(lambda s: s[1] in chosen_indices, filtered_dataset.samples))
filtered_dataset.samples = list(map(lambda s: (s[0], new_class_idx_mapping[s[1]]), filtered_dataset.samples))

filtered_train_subset, filtered_test_subset = torch.utils.data.random_split(filtered_dataset, [0.8, 0.2], torch.Generator().manual_seed(42))

train_dl = torch.utils.data.DataLoader(filtered_train_subset, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(filtered_test_subset, batch_size=64, shuffle=True)

print(len(filtered_dataset))

101000
10000


In [5]:
len(filtered_train_subset)

8000

In [6]:
model.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=512, out_features=num_classes))
for param in model.classifier.parameters():
        param.requires_grad = True

model.num_labels = num_classes

In [7]:
num_epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

train_model(model, train_dl, test_dl, num_epochs, optimizer, early_stopping=early_stopping)

100%|██████████| 32/32 [00:25<00:00,  1.26it/s]


[Epoch 1] Loss: 4.543, Train Acc: 0.614,Valid loss: 1.275 Valid Acc: 0.588


100%|██████████| 32/32 [00:23<00:00,  1.34it/s]


[Epoch 2] Loss: 3.014, Train Acc: 0.745,Valid loss: 1.076 Valid Acc: 0.639


100%|██████████| 32/32 [00:22<00:00,  1.40it/s]


...

[Epoch 4] Loss: 1.645, Train Acc: 0.861,Valid loss: 1.323 Valid Acc: 0.626


100%|██████████| 32/32 [00:23<00:00,  1.33it/s]


[Epoch 5] Loss: 1.121, Train Acc: 0.904,Valid loss: 1.344 Valid Acc: 0.616


100%|██████████| 32/32 [00:24<00:00,  1.32it/s]

[Epoch 6] Loss: 0.793, Train Acc: 0.934,Valid loss: 1.605 Valid Acc: 0.608
Early stopping at epoch 6


In [8]:
torch.save(model.state_dict(), "model_finetuned_baseline.pth")

In [7]:
#finetuned_path = '/content/drive/MyDrive/automating_science/model_finetuned_baseline.pth' #model_finetuned_baseline.pth
finetuned_path = "model_finetuned_baseline.pth"
checkpoint = torch.load(finetuned_path,  map_location=device)
model.load_state_dict(checkpoint)
model = model.to(device)

In [8]:
train_dl_no_shuffle = torch.utils.data.DataLoader(filtered_train_subset, batch_size=32, shuffle=False)
test_dl_no_shuffle = torch.utils.data.DataLoader(filtered_test_subset, batch_size=32, shuffle=False)

In [9]:
class ResNetOutput(modelout_functions.AbstractModelOutput):
    def __init__(self, loss_temperature: float = 1.0):
       super().__init__()
       self.softmax = nn.Softmax(dim=-1)
       self.loss_temperature = loss_temperature

    @staticmethod
    def get_output(
                model: torch.nn.Module,
                weights: Iterable[torch.Tensor],
                buffers: Iterable[torch.Tensor],
                image: torch.Tensor,
                label: torch.Tensor
      ):
      for key, value in weights.items():
        weights[key] = weights[key].to(device)
      output = torch.func.functional_call(model, (weights, buffers), image.unsqueeze(0))
      logits = output.logits #our change
      bindex = torch.arange(logits.shape[0]).to(logits.device, non_blocking=False)
      logits_correct = logits[bindex, label.unsqueeze(0)]

      cloned_logits = logits.clone()

      cloned_logits[bindex, label.unsqueeze(0)] = torch.tensor(-torch.inf, device=logits.device, dtype=logits.dtype)

      margins = logits_correct - cloned_logits.logsumexp(dim=-1)
      return margins.sum()
    
    def get_out_to_loss_grad(self, model, weights, buffers, batch):
      for key, value in weights.items():
        weights[key] = weights[key].to(device)
      images, labels = batch
      output = torch.func.functional_call(model, (weights, buffers), images)
      logits = output.logits #our change

      ps = self.softmax(logits / self.loss_temperature)[torch.arange(logits.size(0)), labels]
      return (1 - ps).clone().detach().unsqueeze(-1)

In [10]:
traker = TRAKer(model=model,
                task=ResNetOutput(),
                train_set_size=len(train_dl_no_shuffle.dataset))

ERROR:TRAK:Could not use CudaProjector.
Reason: No module named 'fast_jl'
ERROR:TRAK:Defaulting to BasicProjector.
INFO:STORE:Existing model IDs in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results: [0]
INFO:STORE:Model IDs that have been finalized: [0]
INFO:STORE:Existing TRAK scores:
INFO:STORE:test_val: C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results\scores\test_val.mmap


In [16]:
model_id = 0
traker.load_checkpoint(checkpoint, model_id=0)

In [17]:
for data in tqdm(train_dl_no_shuffle):
    data = [xy.cuda() for xy in data]

    traker.featurize(batch=data, num_samples=data[0].shape[0])

  0%|          | 0/250 [00:00<?, ?it/s]c:\Users\kamil\anaconda3\envs\ml_gnn\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 250/250 [24:19<00:00,  5.84s/it]


In [18]:
traker.finalize_features()

Finalizing features for all model IDs..: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


In [19]:
traker.start_scoring_checkpoint(exp_name="test_val", checkpoint=checkpoint, model_id=model_id, num_targets=len(test_dl_no_shuffle.dataset))
for batch in tqdm(test_dl_no_shuffle):
    batch = [xy.cuda() for xy in batch]
    traker.score(batch=batch, num_samples=batch[0].shape[0])

test_scores = traker.finalize_scores(exp_name="test_val")

Finalizing scores for all model IDs..: 100%|██████████| 1/1 [00:00<00:00, 11.92it/s]
INFO:STORE:Saving scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results\scores/test_val.mmap


In [20]:
test_scores.shape

(8000, 2000)

In [11]:
import numpy as np

np.save("test_scores.npy", test_scores)

NameError: name 'test_scores' is not defined

In [12]:
test_scores = np.load("test_scores.npy")

In [13]:
targets = []

for _, y in test_dl_no_shuffle:
    targets.extend(y.tolist())
targets = np.array(targets)

In [15]:
def compute_test_dataset_predictions(model, test_dl):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm(test_dl):
            batch = [xy.cuda() for xy in batch]
            output = model(batch[0]).logits
            preds.extend(output.tolist())
    preds = torch.Tensor(preds)
    preds = torch.nn.functional.softmax(preds, dim=-1)
    return np.array(preds.tolist())


preds = compute_test_dataset_predictions(model, test_dl_no_shuffle)

100%|██████████| 63/63 [00:21<00:00,  2.99it/s]


In [16]:
# https://openreview.net/pdf?id=Agekm5fdW3 - section 2.2
def compute_class_weights(model, preds, targets):
    class_weights = []
    for i in range(num_classes):
        n_elements = np.sum(targets == i)
        class_weights.append(np.exp(1 / n_elements * np.sum(np.log(preds[targets == i][:, i]))))
    return np.array(class_weights)

class_weights = compute_class_weights(model, preds, targets)
class_weights

array([0.15793838, 0.27197625, 0.12155634, 0.01027009, 0.27727489,
       0.1507057 , 0.26520708, 0.52252223, 0.46870035, 0.59647187])

In [17]:
# https://openreview.net/pdf?id=Agekm5fdW3 - section 2.2
def compute_group_alignment_scores(test_scores, targets, class_weights):
    group_alignment_scores = []
    n_train_examples = test_scores.shape[0]
    n_targets = []
    for i in range(num_classes):
        n_targets.append(np.sum(targets == i))
    n_targets = np.array(n_targets)
    for i in tqdm(range(n_train_examples)):
        example_score = 0.0
        for j in range(num_classes):
            scaling_factor = class_weights[j] / n_targets[j]
            example_score += scaling_factor * np.sum(test_scores[i, targets == j])
        group_alignment_scores.append(example_score)
    return np.array(group_alignment_scores)

group_alignment_scores = compute_group_alignment_scores(test_scores, targets, class_weights)
(group_alignment_scores < 0).sum()

100%|██████████| 8000/8000 [00:01<00:00, 6015.43it/s]


2884

In [18]:
example_indices_to_keep = np.nonzero(~(group_alignment_scores < 0))[0]
example_indices_to_keep

array([   0,    1,    2, ..., 7996, 7997, 7998], dtype=int64)

In [19]:
train_data_after_trak = torch.utils.data.Subset(filtered_train_subset, example_indices_to_keep)
train_dl_after_trak = torch.utils.data.DataLoader(train_data_after_trak, batch_size=64, shuffle=True)

model_after_trak = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")
model_after_trak.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=512, out_features=num_classes))
for param in model_after_trak.classifier.parameters():
        param.requires_grad = True
model_after_trak.num_labels = num_classes

In [20]:
num_epochs = 20
optimizer = torch.optim.Adam(model_after_trak.parameters(), lr=1e-3)
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

train_model(model_after_trak, train_dl_after_trak, test_dl, num_epochs, optimizer, early_stopping=early_stopping)

100%|██████████| 32/32 [00:19<00:00,  1.61it/s]


[Epoch 1] Loss: 2.747, Train Acc: 0.638,Valid loss: 1.585 Valid Acc: 0.552


100%|██████████| 32/32 [00:19<00:00,  1.61it/s]


[Epoch 2] Loss: 1.633, Train Acc: 0.788,Valid loss: 1.058 Valid Acc: 0.643


100%|██████████| 32/32 [00:19<00:00,  1.61it/s]


[Epoch 3] Loss: 1.273, Train Acc: 0.834,Valid loss: 1.440 Valid Acc: 0.565


100%|██████████| 32/32 [00:22<00:00,  1.45it/s]


[Epoch 4] Loss: 0.868, Train Acc: 0.887,Valid loss: 1.433 Valid Acc: 0.635


100%|██████████| 32/32 [00:43<00:00,  1.35s/it]

[Epoch 5] Loss: 0.538, Train Acc: 0.932,Valid loss: 1.730 Valid Acc: 0.617
Early stopping at epoch 5


In [ ]:
# TODO
# show improved predictions on some examples
# shortcut learning / simplicity bias
# "describe your findings. Please find an example where there is a surprising connection between training data and some prediction."
# maybe choose better k (number of removed examples) - the authors say the heuristic (of choosing everything < 0) results in over-estimated k

# write a few-page summary
# readme, installation documentation
# code cleanup